In [93]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

# Метод на основе 'генераторов'

In [94]:
MyErrorWord = 'NaN'

def myintersect(a, b):
# поиск пересечения по значениям у двух объектов
# returns (intersection list with len(a), count of intersection positions == support) or Error message
    if (len(a) == len(b)):
        result = []
        count = 0
        for i in range(len(a)):
            if (a[i] == b[i]):
                result.append(a[i])
                count += 1
            else:
                result.append(MyErrorWord)
        return result, count 
    else:
        print ('Error(not corresponds dimentions)')
        return 'Error(not corresponds dimentions)', 0
def issub(a, b):
# включается ли один объект(а) в другой(b)
# returns True or False or Error message
    if (len(a) == len(b)):
        result = []
        count = 0
        for i in range(len(a)):
            if (a[i]!= MyErrorWord):
                if (a[i] != b[i]):
                    return False
        return True
    else:
        print ('Error(not corresponds dimentions)')
        return 'Error(not corresponds dimentions)'

positive = []
countPosEx = 0
negative = []
countNegEx = 0
ff = []

#### Считывание размеченной тренировочной выборки

In [95]:
def readtrain(INPUTnum):
    global positive;
    global negative;
    global ff;
    global countPosEx;
    global countNegEx;
    
    positive = []
    countPosEx = 0
    negative = []
    countNegEx = 0
    ff = []
    
    f = open("train"+str(INPUTnum)+".csv", 'r')
    ff = pd.read_csv(f)
    f.close()
    ff = ff.values
    for i in ff:
        if i[-1] == 'positive':
            positive.append(i[:-1])
        else:
            negative.append(i[:-1])

    f = open("test"+str(INPUTnum)+".csv")
    ff = pd.read_csv(f)
    f.close()
    ff = ff.values
    countPosEx = len(positive)
    countNegEx = len(negative)

#### Проверка классифицируемого объекта на плюс-контексте и минус-контексте (получение необходимых оценок, для классификации)

In [96]:
def mycheck(NewObject):
    global positive;
    global negative;
    global ff;
    global countPosEx;
    global countNegEx;
    
    
    #print (NewObject)
    i1 = NewObject[:-1]
    
    # check for g+
    maxPlusSup = 0;
    summPlusSups = 0.0;
    intersectionsPos = []
    for j in positive:
        myflag1 = 0
        T = myintersect(i1, j)
        intersectionsPos.append(T[1])
        if (T[1] > 0):
            if T[1] > maxPlusSup:
                maxPlusSup = T[1]
            summPlusSups += T[1]
            myflag = 0
            for q in negative:
                if issub(T[0], q):
                    myflag = 1
            if (myflag == 0):
                myflag1 = 1
    #print ('+=', myflag1)
    #print ('+=', maxPlusSup, ' avg = ', summPlusSups/countPosEx)
    
    # check for g-
    maxMinusSup = 0;
    summMinusSups = 0.0;
    intersectionsNeg = [];
    for j in negative:
        myflag1 = 0
        T = myintersect(i1, j)
        intersectionsNeg.append(T[1])
        if (T[1] > 0):
            if T[1] > maxMinusSup:
                maxMinusSup = T[1]
            summMinusSups += T[1]
            myflag = 0
            for q in positive:
                if issub(T[0], q):
                    myflag = 1
            if (myflag == 0):
                myflag1 = 1
    #print ('-=', myflag1)
    #print ('-=', maxMinusSup,  ' avg = ', summMinusSups/countNegEx)
    
    return (maxPlusSup, summPlusSups/countPosEx, maxMinusSup, summMinusSups/countNegEx, intersectionsPos, intersectionsNeg)

### Классификатор

#### посмотрим на оценки(и будем использовать их для классификации):
- максимальная поддержка в контексте
- средняя поддержка для всего контекста  
(попробуем поподбирать различные варианты условий, для построения дерева решения  
одним из наилучших будет следующей вариант)

In [97]:
def decide(T):
    if (T[0] > T[2] + 1):
        if (abs(T[1]-T[3]) <= 0.5):
            return 'positive'
        elif T[3]-T[1] > 0.5:
            return 'negative'
        else:
            return 'positive'
        
    elif (T[2] > T[0] + 1):
        if (abs(T[1]-T[3]) <= 0.5):
            return 'negative'
        elif T[1]-T[3] > 0.5:
            return 'positive'
        else:
            return 'negative'
    elif (T[0] == T[2]):
        if T[1] > T[3]:
            return 'positive'
        else:
            return 'negative'
    else:
        if (T[0]*T[1] > T[2]*T[3]):
            return 'positive'
        else:
            return 'negative'

#### Вычисление оценок

In [98]:
def estimation():
    global ff;
    
    result = []
    TP = 0.0 # True positive
    TN = 0.0 # True negative
    FP = 0.0 # False positive
    FN = 0.0 # False negative
    F1 = 0.0
    for i in range(len(ff)):
        r = mycheck(ff[i])
        result.append(decide(r))
        if (ff[i][-1] == 'positive'):
            if result[-1] == ff[i][-1]:
                TP += 1
            else:
                FN += 1
        else:
            if result[-1] == ff[i][-1]:
                TN += 1
            else:
                FP += 1
    return {'TP':TP,'FN':FN, 'FP':FP, 'TN':TN}

#### Проверка всех новых объектов(тестовые выборки)

In [99]:
test = []
for i in range(1,11):
    readtrain(i)
    test.append(estimation())
    print (test[-1]['TP'], test[-1]['FN'])
    print (test[-1]['FP'], test[-1]['TN'])
    # precision
    test[-1]['precision'] = test[-1]['TP']/(test[-1]['TP']+test[-1]['FP'])
    print ('precision=', test[-1]['precision'])
    # recall
    test[-1]['recall'] = test[-1]['TP']/(test[-1]['TP']+test[-1]['FN'])
    print ('recall=', test[-1]['recall'])
    # F1-score
    test[-1]['F1'] = 2/(1/test[-1]['recall']+1/test[-1]['precision'])
    print ('F1=', test[-1]['F1'])

35.0 26.0
8.0 24.0
precision= 0.813953488372093
recall= 0.5737704918032787
F1= 0.673076923076923
37.0 14.0
14.0 22.0
precision= 0.7254901960784313
recall= 0.7254901960784313
F1= 0.7254901960784313
49.0 16.0
8.0 27.0
precision= 0.8596491228070176
recall= 0.7538461538461538
F1= 0.8032786885245903
37.0 22.0
11.0 19.0
precision= 0.7708333333333334
recall= 0.6271186440677966
F1= 0.691588785046729
41.0 21.0
11.0 16.0
precision= 0.7884615384615384
recall= 0.6612903225806451
F1= 0.7192982456140351
39.0 17.0
8.0 21.0
precision= 0.8297872340425532
recall= 0.6964285714285714
F1= 0.7572815533980582
46.0 24.0
13.0 31.0
precision= 0.7796610169491526
recall= 0.6571428571428571
F1= 0.7131782945736435
52.0 21.0
13.0 21.0
precision= 0.8
recall= 0.7123287671232876
F1= 0.753623188405797
56.0 14.0
10.0 23.0
precision= 0.8484848484848485
recall= 0.8
F1= 0.8235294117647058
38.0 21.0
13.0 19.0
precision= 0.7450980392156863
recall= 0.6440677966101694
F1= 0.6909090909090909


In [100]:
p = np.array([i['precision'] for i in test])
r = np.array([i['recall'] for i in test])
f1 = np.array([i['F1'] for i in test])
print ('avg p = ', np.mean(p))
print ('avg r = ', np.mean(r))
print ('avg f1 = ', np.mean(f1))

avg p =  0.796141881774
avg r =  0.685148380068
avg f1 =  0.735125437739


#### Видим, что классификация не очень удачная,  так как среднее значение F1-меры только 0.73

# Теперь попробуем с помощью RandomForestClassifier

In [101]:
from sklearn.ensemble import RandomForestClassifier as RFC

### Сделаем преобразование входных данных с помощью отображения:
- 'x' -> 1
- 'b' -> 0
- 'o' -> -1

In [102]:
def mytransform(a):
    res = []
    for i in range(len(a)):
        if (a[i] == 'x'):
            res.append(1)
        elif (a[i] == 'b'):
            res.append(0)
        else:
            res.append(-1)
    return res

#### Чтение входных данных

In [103]:
def readtrainForest(INPUTnum):
    global positive;
    global negative;
    global ff;
    global countPosEx;
    global countNegEx;
    
    ff = []
    
    f = open("train"+str(INPUTnum)+".csv", 'r')
    ff = pd.read_csv(f)
    #print (ff.head())
    X = ff[['V1','V2','V3','V4','V5','V6','V7','V8','V9']].values
    X_r = []
    for i in X:
        X_r.append(mytransform(i))
    y = ff['V10'].values
    f.close()
    
    f = open("test"+str(INPUTnum)+".csv")
    ff = pd.read_csv(f)
    X1 = ff[['V1','V2','V3','V4','V5','V6','V7','V8','V9']].values
    X1_r = []
    for i in X1:
        X1_r.append(mytransform(i))
    y1 = ff['V10'].values
    f.close()
    
    return X_r, y, X1_r, y1

In [104]:
test_RFC = []
myrfc = RFC()
for i in range(1,11):
    X, y, X1, y1 = readtrainForest(i)
    myrfc.fit(X,y)
    y1_pred = myrfc.predict(X1)
    
    TP = 0.0 # True positive
    TN = 0.0 # True negative
    FP = 0.0 # False positive
    FN = 0.0 # False negative
    F1 = 0.0
    for i in range(len(y1)):
        if (y1[i] == 'positive'):
            if y1[i] == y1_pred[i]:
                TP += 1
            else:
                FN += 1
        else:
            if y1[i] == y1_pred[i]:
                TN += 1
            else:
                FP += 1
    test_RFC.append({'TP':TP,'FN':FN, 'FP':FP, 'TN':TN})
    print (test_RFC[-1]['TP'], test_RFC[-1]['FN'])
    print (test_RFC[-1]['FP'], test_RFC[-1]['TN'])
    # precision
    test_RFC[-1]['precision'] = test_RFC[-1]['TP']/(test_RFC[-1]['TP']+test_RFC[-1]['FP'])
    print ('precision=', test_RFC[-1]['precision'])
    # recall
    test_RFC[-1]['recall'] = test_RFC[-1]['TP']/(test_RFC[-1]['TP']+test_RFC[-1]['FN'])
    print ('recall=', test_RFC[-1]['recall'])
    # F1-score
    test_RFC[-1]['F1'] = 2/(1/test_RFC[-1]['recall']+1/test_RFC[-1]['precision'])
    print ('F1=', test_RFC[-1]['F1'])

60.0 1.0
2.0 30.0
precision= 0.967741935483871
recall= 0.9836065573770492
F1= 0.9756097560975611
49.0 2.0
1.0 35.0
precision= 0.98
recall= 0.9607843137254902
F1= 0.9702970297029702
64.0 1.0
0.0 35.0
precision= 1.0
recall= 0.9846153846153847
F1= 0.9922480620155039
57.0 2.0
3.0 27.0
precision= 0.95
recall= 0.9661016949152542
F1= 0.957983193277311
60.0 2.0
2.0 25.0
precision= 0.967741935483871
recall= 0.967741935483871
F1= 0.9677419354838711
55.0 1.0
0.0 29.0
precision= 1.0
recall= 0.9821428571428571
F1= 0.9909909909909909
70.0 0.0
0.0 44.0
precision= 1.0
recall= 1.0
F1= 1.0
71.0 2.0
1.0 33.0
precision= 0.9861111111111112
recall= 0.9726027397260274
F1= 0.9793103448275863
70.0 0.0
2.0 31.0
precision= 0.9722222222222222
recall= 1.0
F1= 0.9859154929577465
58.0 1.0
0.0 32.0
precision= 1.0
recall= 0.9830508474576272
F1= 0.9914529914529915


In [105]:
p_RFC = np.array([i['precision'] for i in test_RFC])
r_RFC = np.array([i['recall'] for i in test_RFC])
f1_RFC = np.array([i['F1'] for i in test_RFC])
print ('avg p = ', np.mean(p_RFC))
print ('avg r = ', np.mean(r_RFC))
print ('avg f1 = ', np.mean(f1_RFC))

avg p =  0.98238172043
avg r =  0.980064633044
avg f1 =  0.981154979681


#### Видим,  что RFC дает ошибку в 2-4 примера, что почти идеально классифицирует объекты